이번 번역은 앞서 번역했던 Data_Sciencetutorial_for_Beginners에서 Data Visualization부분 중 하나인 Seaborn 부분을 번역하였습니다.
원본은 https://www.kaggle.com/kanncaa1/seaborn-tutorial-for-beginners 를 참고 해주시기 바랍니다.

역자 : 이 커널은 Data Visualization만을 위한 커널이라기보단 이 커널을 해결하기위해 데이터를 Visualization 예를 보여준거같습니다.

# 목차
1. 데이터 읽기
1. 각 주(state)의 빈곤율(poverty rate )
1. 사망한 사람들의 가장 흔한 성과 이름 15개
1. 주(state)에서 25세 이상인 사람들의 고등학교 졸업률
1. 흑인, 백인, 아메리카 원주민, 아시아 및 히스패닉 인종들의 주(state) 인구 비율
1. 고등학교 졸업률 vs 주(state)의 빈곤율
1. 특징 없애기(Kill properties)
    * 사망 방식 (Manner of death)
    * 살인 무기 (Kill weapon)
    * 사망자들의 연령
    * 사망자들의 인종
    * 가장 위험한 도시들
    * 가장 위험한 주(state)들
    * 사망자들이 정신질환을 가지고 있었는지 없었는지.
    * 위협 유형
    * 도망 유형
    * 경찰이 바디캠(body cam)을 가지고 있었는지 없었는지
1. 사망자 데이터에서 주(state)의 인종 비율
1. 사망자 데이터에서 주(state)의 사망자 수
1. Plot 시각화 튜토리얼 : https://www.kaggle.com/kanncaa1/plotly-tutorial-for-beginners
<br>
<br>
Plot Contents:
* [Bar Plot](#1)
* [Point Plot](#2)
* [Joint Plot](#3)
* [Pie Chart](#4)
* [Lm Plot](#5)
* [Kde Plot](#6)
* [Violin Plot](#7)
* [Heatmap](#8)
* [Box Plot](#9)
* [Swarm Plot](#10)
* [Pair Plot](#11)
* [Count Plot](#12)
    


In [16]:
# Python 3 환경에는 많은 유용한 분석 라이브러리들이 설치되어져있다.
# 이것은 kaggle/python 도커(docker) 이미지는 다음 링크에서 받을수 있다. : https://github.com/kaggle/docker-python
# 예를 들어 불러올때 유용한 몇개의 패키지가 있습니다.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
# 입력데이터 파일은 "../input/"에 있습니다.
# 역자: 종료된 커널이다보니 종종 directory 위치가 바뀐 경우가 있다.
# 역자는 따로 file_path 변수를 생성하여 입력 데이터 위치를 맞춰주고자 한다.
# 독자분들께서는 file_path를 자신의 환경에 맞게 수정해주시길 바랍니다.

file_path = 'D:\Chameleon\pytorch\연습\pytorch_exam\DataSet\Fatal Police Shootings in the US'

import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))
# print(check_output(["ls", file_path]).decode("utf8"))

# 현재 디렉토리에 쓰는 모든 결과는 출력으로 저장됩니다.

In [17]:
# data 읽기
median_house_hold_in_come = pd.read_csv(file_path+"/MedianHouseholdIncome2015.csv", encoding="windows-1252")
percentage_people_below_poverty_level = pd.read_csv(file_path + "/PercentagePeopleBelowPovertyLevel.csv", encoding="windows-1252")
percent_over_25_completed_highSchool = pd.read_csv(file_path+'/PercentOver25CompletedHighSchool.csv', encoding="windows-1252")
share_race_city = pd.read_csv(file_path+'/ShareRaceByCity.csv', encoding="windows-1252")
kill = pd.read_csv(file_path+'/PoliceKillingsUS.csv', encoding="windows-1252")

In [18]:
percentage_people_below_poverty_level.head()

,Geographic Area,City,poverty_rate
0,AL,Abanda CDP,78.8
1,AL,Abbeville city,29.1
2,AL,Adamsville city,25.5
3,AL,Addison town,30.7
4,AL,Akron town,42


In [19]:
percentage_people_below_poverty_level.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29329 non-null  object
 1   City             29329 non-null  object
 2   poverty_rate     29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


In [20]:
percentage_people_below_poverty_level.poverty_rate.value_counts()

0       1464
-        201
7.4      129
6.7      129
10.9     128
        ... 
65.9       1
62.3       1
69.1       1
76.5       1
57.2       1
Name: poverty_rate, Length: 771, dtype: int64

In [21]:
percentage_people_below_poverty_level['Geographic Area'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

<a id="1"></a> 
## Bar Plot